# Bert Sentiment Test

In [2]:
import sys
sys.path.append("../../")

In [3]:
import numpy as np
import torch
from IPython.core.display import display, HTML
from transformers import BertTokenizer

from modules.lrp_bert_modules import LRPBertForSequenceClassification

from visualization.heatmap import html_heatmap

ModuleNotFoundError: No module named 'interpret_nlp'

In [48]:
from torch import nn

AttributeError: module 'interpret_nlp.modules' has no attribute 'visualization'

Load model

In [3]:
# print("Loading model...")
# config_path = "bert-sst-config.pt"
# state_dict_path = "bert-sst.pt"

# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
# model = LRPBertForSequenceClassification(torch.load(config_path))
# model.load_state_dict(torch.load(state_dict_path))
# model.eval()
# print("Done.")

Loading model...
Done.


In [ ]:
# CIURRENTLY THIS IS WORKING WITH ORIGINAL MODEL FROM CLINICAL BERT

In [40]:
config_path = "./bert_config.json"
model_path = "./pytorch_model.bin"
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = LRPBertForSequenceClassification.from_pretrained("./", num_labels = 1)
# model = LRPBertForSequenceClassification.from_pretrained(torch.load(model_path))

# model = torch.load(model_path)
# print(model)


<bound method Module.state_dict of LRPBertForSequenceClassification(
  (bert): LRPBertModel(
    (embeddings): LRPBertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LRPLayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): LRPBertEncoder(
      (layer): ModuleList(
        (0): LRPBertLayer(
          (attention): LRPBertAttention(
            (self): LRPBertSelfAttention(
              (query): LRPLinear(in_features=768, out_features=768, bias=True)
              (key): LRPLinear(in_features=768, out_features=768, bias=True)
              (value): LRPLinear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): LRPBertSelfOutput(
              (dense): LRPLinear(in_features=768, out_features=768, 

Set up example

In [43]:
# test_example = "It's a lovely film with wonderful performances by Buy and " \
#                "Accorsi."

test_example = ' he has experienced acute on chronic diastolic heart failure in the setting of volume overload due to his sepsis.'

In [44]:
test_example

' he has experienced acute on chronic diastolic heart failure in the setting of volume overload due to his sepsis.'

Run normal forward pass

In [49]:
model.eval()
inputs = tokenizer(test_example, return_tensors="pt")
logits = model(**inputs).logits.squeeze()

m = nn.Sigmoid()
logits = torch.squeeze(m(logits)).detach().cpu().numpy()

print(logits)
# classes = ["<unk>", "positive", "negative", "neutral"]
classes = ["not_admitted", "admitted"]

# print("Logit Scores:")
# for c, score in zip(classes, logits):
#     print("{}: {}".format(c, score))

0.8606415


Run attribution forward pass

In [50]:
inputs = tokenizer(test_example, return_tensors="pt")
model.attr()
output = model(**inputs)

print("Attr Forward Pass Output:")
print(output)

Attr Forward Pass Output:
[[1.8206284]]


Run LRP

In [56]:
tokens = tokenizer.tokenize(test_example)
rel_y = np.zeros(output.shape)
rel_y[:, 0] = output[:, 0]
rel_word, rel_pos, rel_type, rel_embed = model.attr_backward(rel_y, eps=.1)
rel_word = np.sum(rel_word[0, 1:-1], -1)
rel_pos = np.sum(rel_pos[0, 1:-1], -1)
rel_type = np.sum(rel_type[0, 1:-1], -1)
rel_embed = np.sum(rel_embed[0, 1:-1], -1)

print("LRP Scores:")
for t, s in zip(tokens, rel_embed):
    print(t, s, sep=": ")
    
print("Relevance of word embeddings:")
display(HTML(html_heatmap(tokens, list(rel_word))))

print("Relevance of positional embeddings:")
display(HTML(html_heatmap(tokens, list(rel_pos))))

print("Relevance of type embeddings:")
display(HTML(html_heatmap(tokens, list(rel_type))))

print("Relevance of combined embeddings:")
display(HTML(html_heatmap(tokens, list(rel_embed))))

LRP Scores:
he: 0.023229010748793775
has: 0.009376843853490531
experienced: 0.0030572293401592556
acute: 0.016233898386067237
on: 0.0022083858891211484
chronic: 0.024892846885231558
dia: 0.0006336948378041895
##sto: 0.006915873119135614
##lic: 0.005015703820084047
heart: 0.017993832390379483
failure: 0.0218779687720887
in: -0.0008363484028907171
the: 0.002043464432778934
setting: -0.017114741092442196
of: -0.005795778446079742
volume: 0.00019336913559048446
over: 0.003116229516476183
##load: -0.013129035339667248
due: 0.024921277787359534
to: 0.004506036147102576
his: -0.011142785601670976
sep: 0.0022926451238911
##sis: 0.033563854029015266
.: 0.0018885693068613082
Relevance of word embeddings:


Relevance of positional embeddings:


Relevance of type embeddings:


Relevance of combined embeddings:


In [55]:
def get_LRP_derived_token_scores():
    

IndexError: index 1 is out of bounds for axis 1 with size 1